In [7]:
from keras.layers import Input,Lambda,Dense,Flatten
from keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator,load_img
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt


In [21]:
Img_size= [224,224] +[3]

train_path='./Datasets/train'
test_path= './Datasets/test'



In [22]:
resnet= ResNet50(input_shape= Img_size, weights='imagenet',include_top=False)

In [23]:
resnet.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_4[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [35]:
#Dont train existing weights

for layer in resnet.layers:
    layer.trainable= False

In [40]:
folders= glob(train_path+'/*')
folders

['./Datasets/train\\audi',
 './Datasets/train\\lamborghini',
 './Datasets/train\\mercedes']

In [45]:
x= Flatten()(resnet.output)

In [46]:
prediction=Dense(len(folders), activation='softmax')(x)


In [49]:
#Create Model object

model=Model(inputs=resnet.input, outputs= prediction)

In [52]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_4[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [53]:
# Compiling the model

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [55]:
#read All the data- with some Image/Data augmentation

train_datagen= ImageDataGenerator(rescale=1.0/255)
test_datagen= ImageDataGenerator(rescale=1.0/255)

In [56]:
train_data= train_datagen.flow_from_directory(train_path,target_size= (224,224),batch_size=32,class_mode='categorical')
test_data= test_datagen.flow_from_directory(test_path,target_size=(224,224),batch_size=32,class_mode='categorical')

Found 64 images belonging to 3 classes.
Found 58 images belonging to 3 classes.


In [153]:
model.fit(train_data, validation_data=test_data,epochs=20,validation_steps=11) ### If saved in any var, do >> var.history()

Epoch 1/20
2/2 [==============================] - 15s 10s/step - loss: 0.1898 - accuracy: 0.9219 - val_loss: 0.9499 - val_accuracy: 0.5862
Epoch 2/20
2/2 [==============================] - 8s 4s/step - loss: 0.2068 - accuracy: 0.9531
Epoch 3/20
2/2 [==============================] - 7s 4s/step - loss: 0.1469 - accuracy: 0.9844
Epoch 4/20
2/2 [==============================] - 8s 4s/step - loss: 0.1603 - accuracy: 0.9531
Epoch 5/20
2/2 [==============================] - 7s 3s/step - loss: 0.1470 - accuracy: 0.9531
Epoch 6/20
2/2 [==============================] - 7s 3s/step - loss: 0.1231 - accuracy: 0.9688
Epoch 7/20
2/2 [==============================] - 7s 3s/step - loss: 0.1257 - accuracy: 0.9844
Epoch 8/20
2/2 [==============================] - 9s 4s/step - loss: 0.1211 - accuracy: 1.0000
Epoch 9/20
2/2 [==============================] - 8s 4s/step - loss: 0.1108 - accuracy: 0.9844
Epoch 10/20
2/2 [==============================] - 7s 3s/step - loss: 0.1055 - accuracy: 0.9688
Epoch

###  Save Model

In [154]:
from keras.models import load_model

In [155]:
model.save('model_resnet50.h5')

###  Load Model

In [156]:
model= load_model('model_resnet50.h5')

###  Predict Test Images

In [157]:
y_pred= model.predict(test_data)

In [158]:
y_pred

array([[1.19680569e-01, 7.44132996e-01, 1.36186510e-01],
       [4.45049442e-02, 7.43135095e-01, 2.12359890e-01],
       [1.67580228e-03, 9.94286239e-01, 4.03802516e-03],
       [3.74427158e-03, 9.92695928e-01, 3.55990650e-03],
       [1.85246242e-03, 9.90046561e-01, 8.10096320e-03],
       [5.32163858e-01, 3.62933457e-01, 1.04902714e-01],
       [3.44921172e-01, 4.29310232e-01, 2.25768596e-01],
       [3.33185382e-02, 8.33050907e-01, 1.33630514e-01],
       [2.48907525e-02, 9.32809770e-01, 4.22994532e-02],
       [5.29628992e-03, 9.82555032e-01, 1.21487277e-02],
       [1.57994255e-01, 3.80502224e-01, 4.61503446e-01],
       [8.97845195e-04, 9.67831584e-05, 9.99005377e-01],
       [4.06330004e-02, 1.64823204e-01, 7.94543743e-01],
       [1.61844604e-02, 7.52159059e-01, 2.31656447e-01],
       [2.42178459e-02, 9.62882161e-01, 1.29000023e-02],
       [5.62651530e-02, 5.02629936e-01, 4.41104949e-01],
       [9.87796366e-01, 4.95576393e-03, 7.24783074e-03],
       [6.08404283e-04, 9.67265

In [159]:
y_pred= np.argmax(y_pred, axis=1)

In [160]:
y_pred[3]

1

In [161]:
y_pred

array([1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 0, 1, 1, 1, 0, 1,
       0, 2, 1, 1, 2, 1, 2, 1, 1, 1, 0, 2, 2, 2, 1, 1, 1, 1, 0, 1, 2, 0,
       1, 2, 1, 1, 1, 0, 1, 2, 2, 2, 1, 2, 1, 2], dtype=int64)

In [162]:
res=[]
for i in range(len(y_pred)):
    if y_pred[i]==0: res.append('audi')
    elif y_pred[i]==1: res.append('lamborgini')
    if y_pred[i]==2: res.append('mercedes')

In [163]:
res

['lamborgini',
 'lamborgini',
 'lamborgini',
 'lamborgini',
 'lamborgini',
 'audi',
 'lamborgini',
 'lamborgini',
 'lamborgini',
 'lamborgini',
 'mercedes',
 'mercedes',
 'mercedes',
 'lamborgini',
 'lamborgini',
 'lamborgini',
 'audi',
 'lamborgini',
 'lamborgini',
 'lamborgini',
 'audi',
 'lamborgini',
 'audi',
 'mercedes',
 'lamborgini',
 'lamborgini',
 'mercedes',
 'lamborgini',
 'mercedes',
 'lamborgini',
 'lamborgini',
 'lamborgini',
 'audi',
 'mercedes',
 'mercedes',
 'mercedes',
 'lamborgini',
 'lamborgini',
 'lamborgini',
 'lamborgini',
 'audi',
 'lamborgini',
 'mercedes',
 'audi',
 'lamborgini',
 'mercedes',
 'lamborgini',
 'lamborgini',
 'lamborgini',
 'audi',
 'lamborgini',
 'mercedes',
 'mercedes',
 'mercedes',
 'lamborgini',
 'mercedes',
 'lamborgini',
 'mercedes']

### Loading the Image

In [164]:
def predict(img):
    x= image.img_to_array(img)
    x=x/255
    x=np.expand_dims(x, axis=0)
    result= model.predict(x) 
    result= np.argmax(result)
    if result==0: return 'audi'
    elif result==1: return 'lamborgini'
    if result==2: return 'mercedes'

In [170]:
predict(image.load_img('audi1.jpg', target_size= (224,224)))

'audi'

In [171]:
predict(image.load_img('audi2.jpg', target_size=(224,224)))

'mercedes'

In [172]:
predict(image.load_img('lam1.jpg', target_size=(224,224)))

'lamborgini'

In [173]:
predict(image.load_img('lam2.jpg', target_size=(224,224)))

'mercedes'

In [174]:
predict(image.load_img('mer1.jpg', target_size=(224,224)))

'audi'

In [176]:
import os

In [177]:
basepath= os.path.dirname(__file__)

NameError: name '__file__' is not defined